In [23]:
import sys
from pathlib import Path
import pandas as pd

# Add project root to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import project paths
from src.helpers.config import DATA_DIR
from src.helpers.utils import (
    get_files_paths,
    filter_dataframe,
    analyze_dataframe,
    write_csv,
    fill_missing_values_based_on_column_mapping,
)

# Load dataframe

In [24]:
dataset_name = "teleporady"

In [25]:
file_paths = get_files_paths(Path(DATA_DIR / "raw"), dataset_name, "csv")
file_paths

['c:\\Users\\huber\\OneDrive\\Dokumenty\\GitHub\\mz_raport\\data\\raw\\teleporady\\teleporady.csv']

In [26]:
file_path = file_paths[0]
file_path

'c:\\Users\\huber\\OneDrive\\Dokumenty\\GitHub\\mz_raport\\data\\raw\\teleporady\\teleporady.csv'

In [27]:
# Read only the first 10 rows to inspect column names
df_preview = pd.read_csv(file_path, nrows=2)
df_preview

,Nazwa świadczeniodawcy,Województwo,Powiat,Gmina,Specjalność komórki,Miesiąc,Rok,Kod ICD-10 poziom 3.,Nazwa ICD-10 poziom 3.,Kod ICD-10 poziom 2.,Nazwa ICD-10 poziom 2.,Kod ICD-10 poziom 1.,Nazwa ICD-10 poziom 1.,Liczba wszystkich porad AOS,Liczba teleporad AOS
0,105 Kresowy Szpital Wojskowy Z Przychodnią sp....,dolnośląskie,bolesławiecki,Osiecznica,poradnia położniczo-ginekologiczna,1,2021,D25,Mięśniak gładkokomórkowy macicy,D10-D36,Nowotwory niezłośliwe,C00-D48,Nowotwory,1,0
1,105 Kresowy Szpital Wojskowy Z Przychodnią sp....,dolnośląskie,bolesławiecki,Osiecznica,poradnia położniczo-ginekologiczna,1,2021,N30,Zapalenie pęcherza moczowego,N30-N39,Inne choroby układu moczowego,N00-N99,Choroby układu moczowo-płciowego,1,0


In [28]:
# Process large CSV file in chunks and merge filtered parts
chunk_size = 500000  # Number of rows to load at a time
col = "Specjalność komórki"
speciality = "poradnia okulistyczna"
filter_dict = {col: speciality}

filtered_dataframes = []
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # Filter the data based on the provided criteria
    filtered_chunk = filter_dataframe(chunk, filter_dict=filter_dict)

    if not filtered_chunk.empty:
        filtered_dataframes.append(filtered_chunk)

# Merge all filtered chunks into a single DataFrame
df_raw = pd.concat(filtered_dataframes, ignore_index=True)


# Analyze dataframe

In [29]:
df_raw.shape

(758245, 15)

In [30]:
analyze_dataframe(df_raw)

,column_name,missing_values_total,missing_values_percent,unique_values_count,data_type
0,Nazwa świadczeniodawcy,0,0.000000,1494,object
1,Województwo,198,0.026113,16,object
2,Powiat,0,0.000000,349,object
3,Gmina,198,0.026113,617,object
4,Specjalność komórki,0,0.000000,1,object
5,Miesiąc,0,0.000000,12,int64
6,Rok,0,0.000000,4,int64
7,Kod ICD-10 poziom 3.,5,0.000659,999,object
8,Nazwa ICD-10 poziom 3.,5,0.000659,999,object
9,Kod ICD-10 poziom 2.,5,0.000659,226,object


## 198 rows with missing values in Województwo

In [31]:
df = df_raw.copy()

In [33]:
df[df["Województwo"].isna() & df["Powiat"].notna()].shape


(198, 15)

In [34]:
df = fill_missing_values_based_on_column_mapping(
    df, target_col="Województwo", reference_col="Powiat"
)

In [35]:
df[df["Województwo"].isna()].shape

(0, 15)

# Save processed dataframe

In [36]:
file_path = Path(
    DATA_DIR / "processed" / f"{dataset_name}_{speciality.replace(' ', '_')}.csv"
)
file_path

WindowsPath('c:/Users/huber/OneDrive/Dokumenty/GitHub/mz_raport/data/processed/teleporady_poradnia_okulistyczna.csv')

In [37]:
write_csv(df, file_path)

CSV file successfully written to: c:\Users\huber\OneDrive\Dokumenty\GitHub\mz_raport\data\processed\teleporady_poradnia_okulistyczna.csv
